In [3]:
import random
import requests
from PIL import Image
from base64 import b64encode
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import time
from urllib import *
import urllib
import json

client_id = "e291792abd4b4a1db646f561c823beb9"
client_secret = "26e6df1ae3614825940739dd03c31feb"

auth_url = 'https://accounts.spotify.com/api/token'
auth_header = 'Basic ' + b64encode((client_id + ':' + client_secret).encode()).decode()

auth_data = {
    'grant_type': 'client_credentials'
}

auth_response = requests.post(auth_url, headers={'Authorization': auth_header}, data=auth_data)

if auth_response.status_code == 200:
    token = auth_response.json().get('access_token')
    #print('Token:', token)
else:
    print('Error:', auth_response.status_code)
    print(auth_response.text)
    
headers = {
    "Authorization": "Bearer " + token
}

'''def get_artistId():
    print(get_artists["id"])
    
def get_artistName():
    print(get_artists["name"])'''

def get_artists(genre, artist):
    genre = genre.lower()
    url = f"https://api.spotify.com/v1/search?q={artist}&type=artist"

    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        artists_info = response.json()
        if artists_info["artists"]["items"] == []:
            print("No artists found for that genre!")
            return []
        else:
            for i in artists_info["artists"]["items"]:
                try:
                    if genre in i["genres"] and i["name"] == artist:
                        return i["id"]
                except IndexError:
                    print("Enter a valid genre or try a different artist!")
                    return []
    else:
        print("Enter a valid genre or try a different artist!")
        return []
    
def get_albums(genre, artist, start_year, end_year):
    while True:
        try:
            # artist_items = get_artists(genre, artist)
            artist_id = get_artists(genre, artist)
        except IndexError:
            return []
    
        response = requests.get(f"https://api.spotify.com/v1/artists/{artist_id}/albums", headers=headers)
        
        artist_albums = response.json()
        
        if response.status_code == 200:
            if len(artist_albums["items"]) < 3:
                print("Not enough albums found for this artists!")
                return []
            else:
                if "items" in artist_albums:
                    lst = []
                    for i in artist_albums["items"]:
                        date_list = list(i["release_date"])
                        year1 = date_list[0]
                        year2 = date_list[1]
                        year3 = date_list[2]
                        year4 = date_list[3]
                        release_year = f"{year1}{year2}{year3}{year4}"
            
                        if int(release_year) in range(int(start_year), int(end_year) + 1):
                            lst += [i]
                    try:
                        rand_albums = random.sample(lst, k=3)
                    except IndexError:
                        print("This time period had less than 3 albums!")
                        return []
                    return rand_albums
        else:
            print("No albums found for this artist or not a valid artist!\n")
            return []

API_KEY = "OdOTpaxGkQyamSaW2EkeanimV3vOKKdmyBTwQHGHxErCN97F3c1Z4kVR8UJWgM85"
HEADERS = {"Authorization": f"Bearer {API_KEY}"}

def get_lyrics(artist):
    response = requests.get("https://api.genius.com/search", headers=HEADERS, params={"q": artist})
    data = response.json()

    if not data["response"]["hits"]:
        print("No songs found.\n")
        return []
    else:
        song = random.choice(data["response"]["hits"])["result"]
        
        title = song["title"]
        
        hint = song["title"][0]

        page = requests.get(song["url"])
        soup = BeautifulSoup(page.text, "html.parser")

        lyrics_divs = soup.find_all("div", class_=lambda x: x and "Lyrics__Container" in x)

        full_lyrics = ""
        
        for i in lyrics_divs:
            full_lyrics += i.text.strip()

        words = full_lyrics.split() 

        if len(words) >= 10:
            stop = random.randint(10, len(words))
            start = stop - 10
            consecutive_words = words[start:stop]
            song_lst = [title, " ".join(consecutive_words), hint]
            return song_lst
                
        else:
            print("Not enough words in the lyrics.\n")
            return []

'''get_lyrics("Taylor Swift")'''

'''client_id = "fe26696907e44c8393d68a7f04883f8c"
client_secret = "eb52c38e47504474adee24d6232b24db"

auth_url = 'https://accounts.spotify.com/api/token'
auth_header = 'Basic ' + b64encode((client_id + ':' + client_secret).encode()).decode()

auth_data = {
    'grant_type': 'client_credentials'
}

auth_response = requests.post(auth_url, headers={'Authorization': auth_header}, data=auth_data)

if auth_response.status_code == 200:
    token = auth_response.json().get('access_token')
    ###print('Token:', token)
else:
    print('Error:', auth_response.status_code)
    print(auth_response.text)


id = "1Xyo4u8uXC1ZmMpatF05PJ" #weeknd
#id = "3TVXtAsR1Inumwj472S9r4" drake
#id = "40ZNYROS4zLfyyBSs2PGe2"  zach bryan
#id = "6KImCVD70vtIoJWnq6nGn3" #harry styles
url = f"https://api.spotify.com/v1/artists/{id}"
url_al = f"https://api.spotify.com/v1/artists/{id}/albums"


headers = {
    "Authorization": "Bearer " + token
}

response = requests.get(url, headers=headers)
response_album = requests.get(url_al,headers=headers)
data = response.json()
alb_data = response_album.json()
name = data['name']
max = len(alb_data['items'])
artist_id = data['id']'''

def image(id):
    url = f"https://api.spotify.com/v1/artists/{id}"
    url_al = f"https://api.spotify.com/v1/artists/{id}/albums"
    response = requests.get(url, headers=headers)
    response_album = requests.get(url_al,headers=headers)
    
    if response.status_code != 200 or response_album.status_code != 200:
        print("Error fetching artist or album data.")
        return []
    
    data = response.json()
    name = data['name']
    alb_data = response_album.json()
    max = len(alb_data['items'])
    ran = random.randint(0,max-1)
    image_url = alb_data['items'][ran]['images'][0]['url']
    
    if max < 1:
        print('No album found')
        return []
    else:
        image_name = alb_data['items'][ran]['name']
        urllib.request.urlretrieve(image_url, f'{image_name}.jpg')
        img = Image.open(f'{image_name}.jpg')
        return [img, image_name]

'''def artist_name():
    name = data['name']
    print(f"{name}")

def artist_id():
    id = data['id']
    return id

def artist_follow():
    name = data['name']
    folow = data['followers']['total']
    print (f"{name} has {folow} followers on spotify ")'''
    
def get_spotify_id(artist_name):
    url = "https://api.spotify.com/v1/search"
    headers = {
        "Authorization": f"Bearer {token}"
    }
    params = {
        "q": artist_name,
        "type": "artist",
        "limit": 1  
    }

    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        results = response.json()
        if results["artists"]["items"]:
            artist = results["artists"]["items"][0]
            id = artist["id"]
            return id
        else:
            return "No artist found."
    else:
        return f"Error: {response.status_code} - {response.text}"
    
def main():
    print("Welcome to our spotify guessing game! You can guess albums in a certain genre, guess the song lyrics from a certain artist, or guess an album title from its cover.")
    response_1 = ""
    while True:
        response_1 = input("What would you like to do? Enter A to guess an album, enter B to guess a song title based on lyrics, or enter C to guess an album title from its cover.")
        if response_1.lower() == "a":
            score = 0
            
            response_4 = input("Enter a genre to guess an album from: ")
            response_15 = input("Enter an artist's first and last name: ")
            response_16 = input("Enter start year: ")
            response_17 = input("Enter end year: ")
            
            album_Q1 = get_albums(response_4, response_15, response_16, response_17)
            
            if album_Q1 == []:
                continue
  
            answer = random.choice(album_Q1)
            
            date_list = list(answer["release_date"])
            year1 = date_list[0]
            year2 = date_list[1]
            year3 = date_list[2]
            year4 = date_list[3]
            
            print(f"\nThis album was released in {year1}{year2}{year3}{year4}\n")
            print(f"\n1. {album_Q1[0]["name"]}\n2. {album_Q1[1]["name"]}\n3. {album_Q1[2]["name"]}\n")
      
            hint = list(answer["name"])[0]

            response_6 = input(f"Guess the album from the list above: ")
            if response_6.lower() == answer["name"].lower():
                score += 10
                print(f"Correct! The album was {answer["name"]}. Your score is {score}/30.")
                
            else:
                if score >= 10:
                    score -= 10
                print(f"Incorrect. The correct album was {answer["name"]}. Your score is {score}/30.\n")
                
                
            album_Q2 = get_albums(response_4, response_15, response_16, response_17)
            
            if album_Q2 == []:
                continue
  
            answer2 = random.choice(album_Q2)
            
            date_list2 = list(answer2["release_date"])
            year5 = date_list2[0]
            year6 = date_list2[1]
            year7 = date_list2[2]
            year8 = date_list2[3]
            
            print(f"\nThis album was released in {year5}{year6}{year7}{year8}\n")
            print(f"\n1. {album_Q2[0]["name"]}\n2. {album_Q2[1]["name"]}\n3. {album_Q2[2]["name"]}\n")
      
            hint2 = list(answer2["name"])[0]

            response_21 = input(f"Guess the album from the list above: ")
            if response_21.lower() == answer2["name"].lower():
                score += 10
                print(f"Correct! The album was {answer2["name"]}. Your score is {score}/30.")
                
            else:
                if score >= 10:
                    score -= 10
                print(f"Incorrect. The correct album was {answer2["name"]}. Your score is {score}/30.\n")
                
            album_Q3 = get_albums(response_4, response_15, response_16, response_17)
            
            if album_Q3 == []:
                continue
  
            answer3 = random.choice(album_Q3)
            
            date_list3 = list(answer3["release_date"])
            year9 = date_list3[0]
            year10 = date_list3[1]
            year11 = date_list3[2]
            year12 = date_list3[3]
            
            print(f"\nThis album was released in {year9}{year10}{year11}{year12}\n")
            print(f"\n1. {album_Q3[0]["name"]}\n2. {album_Q3[1]["name"]}\n3. {album_Q3[2]["name"]}\n")
      
            hint3 = list(answer3["name"])[0]

            response_22 = input(f"Guess the album from the list above: ")
            if response_22.lower() == answer3["name"].lower():
                score += 10
                print(f"Correct! The album was {answer3["name"]}. Your score is {score}/30.")
                
            else:
                if score >= 10:
                    score -= 10
                print(f"Incorrect. The correct album was {answer3["name"]}. Your score is {score}/30.\n")
                
            return f"Your score is {score}/30."
        elif response_1.lower() == "b":
            score = 0
            
            #question 1
            response_2 = input("Enter an artist's name: ")
            question_1 = get_lyrics(response_2)
            
            if question_1 == []:
                continue
            
            response_3 = input(f"Enter the song title for the following lyrics: {question_1[1]}\n")
            if response_3.lower() == question_1[0].lower():
                score += 10
                print(f"\nCorrect! Your score is {score}/30.\n")
            else:
                print(f"\nIncorrect. The correct answer was {question_1[0]}. Your current score is {score}/30\n")
            
            #question 2
            question_2 = get_lyrics(response_2)
            response_5 = input(f"Enter the song title for the following lyrics: {question_2[1]}\n")
            if response_5.lower() == question_2[0].lower():
                score += 10
                print(f"\nCorrect! Your score is {score}/30.\n")
            else:
                print(f"\nIncorrect. The correct answer was {question_2[0]}. Your current score is {score}/30\n")
            
            #question 3
            question_3 = get_lyrics(response_2)
            response_7 = input(f"Enter the song title for the following lyrics: {question_3[1]}\n")
            if response_7.lower() == question_3[0].lower():
                score += 10
                print(f"\nCorrect! Your score is {score}/30.\n")
            else:
                print(f"\nIncorrect. The correct answer was {question_3[0]}. Your current score is {score}/30\n")
            
            return f"Your score is {score}/30."
        
        elif response_1.lower() == "c":
            response_10 = input("Enter an artist to guess the title of their album based on the cover: ")
            spotify_id = get_spotify_id(response_10)
            
            if spotify_id == []:
                continue
            
            album_cover = image(spotify_id)
            hint = album_cover[1][0]
            album_cover[0].show()
            response_11 = input("Guess the album title: ")
            if response_11.lower() == album_cover[1].lower():
                print(f"\nCorrect! The album was {album_cover[1]}")
            else:
                response_12 = input("Guess again: ")
                print(f"The album begins with {hint}\n")
                
                if response_12.lower() == album_cover[1].lower():
                    return f"\nCorrect! The album was {album_cover[1]}"
                else:
                    response_13 = input("Guess one more time: ")
                    if response_13.lower() == album_cover[1].lower():
                        return f"\nCorrect! The album was {album_cover[1]}"
                    else:
                        return f"\nIncorrect. The album was {album_cover[1]}"
        else:
            print("Enter a letter from the option given\n")
            print(response_1)

main()

SyntaxError: f-string: unmatched '[' (425600483.py, line 275)